In [1]:
search_for = 129
start_from = 0
threads = 10
thread = 4
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.10747408866882324
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 423524321
type: [1, 1, 1, 1, 129] 129
cases of this type: 276922881
100000 17.937669734022634
200000 30.395949312686174
300000 31.55662976608057
400000 30.924654740974134
500000 31.309731941440365
600000 32.31514104291248
700000 31.499434827183237
800000 30.546542495524694
900000 30.721741980266277
1000000 30.607805522882654
1100000 13.72977888255761
1200000 27.603435801295433
1300000 30.426365498709906
1400000 30.5334883619546
1500000 30.499400846225598
1600000 30.79009794905973
1700000 32.34676449015403
1800000 30.3469110365902
1900000 30.657825410588973
2000000 31.18165004562961
2100000 30.718709562709254
2200000 8.381522968677155
2300000 35.21106668765375
2400000 38.61672354048966
2500000 38.005503266372735
2600000 37.225439275279015
2700000 37.92319410171652
2800000 35.31441976717046
2900000 35.83898437857525
3000000 35.9565

29600000 29.955807458787245
29700000 31.384419892194174
29800000 30.447270916516207
29900000 17.3302684713566
30000000 32.10829235167284
30100000 32.473227358900154
30200000 31.834848525357774
30300000 21.20648286389635
30400000 23.568592461936223
30500000 33.68843282270036
30600000 32.383052797215164
30700000 33.0126479268686
30800000 32.97940739163523
30900000 29.94751589405775
31000000 30.943517240643935
31100000 27.063992358078234
31200000 13.32244456088653
31300000 30.768390440622795
31400000 29.196856992258116
31500000 30.833922072916632
31600000 32.04809928730003
31700000 31.495505308446344
31800000 34.82874958593284
31900000 30.623150028806332
32000000 13.719238994202655
32100000 34.22235915644584
32200000 31.592667674087874
32300000 34.44216091732831
32400000 30.858189755386686
32500000 12.981517648396247
32600000 34.255138531367805
32700000 31.729951939838728
32800000 33.7827323693124
32900000 29.586713761894686
33000000 30.41619645852949
33100000 30.87632152354965
33200000 2

59400000 26.999717124933834
59500000 26.482109805383686
59600000 28.799880236247215
59700000 12.242146103100584
59800000 30.261569047327363
59900000 29.887739301476365
60000000 29.498132802671652
60100000 30.780141558216958
60200000 30.49554260677972
60300000 30.595092669024194
60400000 29.95701053764315
60500000 30.07862586301065
60600000 16.460530679674257
60700000 24.547512031058538
60800000 27.953426046127277
60900000 27.31896003584537
61000000 27.400485476877186
61100000 26.65173435453646
61200000 13.846870409730576
61300000 23.366990103385298
61400000 26.578344017075445
61500000 26.602193533650837
61600000 26.52118505755853
61700000 27.431135841803982
61800000 16.209654401746832
61900000 25.912912288148693
62000000 29.0242939607431
62100000 30.08821187937907
62200000 29.60304638220351
62300000 28.840799466449923
62400000 29.987419819331997
62500000 28.45424420802483
62600000 29.32140231969203
62700000 28.828894902549496
62800000 10.420667153206297
62900000 26.95891214667662
63000

89100000 13.345997112152656
89200000 19.689661063983337
89300000 23.13548445219097
89400000 24.04081774848817
89500000 12.154313686190674
89600000 22.50145343546079
89700000 23.667896901457187
89800000 24.16678733878906
89900000 24.358814786744258
90000000 26.04857985489341
90100000 25.557582210788024
90200000 26.69131312830246
90300000 25.052742610138157
90400000 27.573989316817947
90500000 25.49140424076217
90600000 24.66708657932212
90700000 24.352682933437418
90800000 22.453322821707935
90900000 11.455881050243496
91000000 21.83728844054982
91100000 23.786889959312152
91200000 23.083660862671277
91300000 9.600907117291127
91400000 23.570280228160684
91500000 22.767774039419848
91600000 17.5242611441837
91700000 18.23316567683906
91800000 22.010500254238973
91900000 23.729979586085722
92000000 23.79226740207212
92100000 25.01775945655096
92200000 26.046262071794448
92300000 24.1426177084871
92400000 26.677048764607456
92500000 25.009617221950748
92600000 24.810244171888126
92700000 

118200000 25.371016313332095
118300000 23.821789226962434
118400000 21.657210577000345
118500000 23.24501429291516
118600000 21.44649308996044
118700000 22.30881366406037
118800000 22.88957156713363
118900000 21.666863009514692
119000000 13.779892684448534
119100000 17.60701006625653
119200000 7.171238051225465
119300000 14.849562819524824
119400000 16.84705453203207
119500000 22.738275629234334
119600000 20.98924340560585
119700000 22.68011216897055
119800000 23.09444478766739
119900000 22.126671569597065
120000000 23.463613252633106
120100000 22.768293094583036
120200000 25.28307114145629
120300000 24.774204340566225
120400000 24.001907565351544
120500000 23.437078066414568
120600000 22.012811289536728
120700000 22.870075938314283
120800000 22.861190645631684
120900000 21.495653495854583
121000000 23.42936361303194
121100000 20.90390288849434
121200000 8.904244729598135
121300000 13.160058552661138
121400000 14.130450073939546
121500000 13.217262265374536
121600000 20.609509562830294

...

290500000 5.172426777885544
290600000 5.745345702320248
290700000 4.833697737367715
290800000 3.593170019842944
290900000 1.1213440344873824
291000000 1.1302289498471882
291100000 5.78187014616261
291200000 5.191819725541778
291300000 5.485102873791128
291400000 5.197021069549589
291500000 4.885456996320897
291600000 5.106507402544461
291700000 5.2958374777236745
291800000 5.2541093376020305
291900000 4.975191096105641
292000000 6.147189818639177
292100000 6.047876541562162
292200000 5.251503036815616
292300000 5.1795967722822915
292400000 4.661590441707885
292500000 5.880419524013972
292600000 4.960213263647842
292700000 5.220162232166212
292800000 4.742215734920675
292900000 5.502489817025615
293000000 1.1347759026435953
293100000 1.1357257688516331
293200000 4.236153253512049
293300000 6.146875871463347
293400000 6.199923131540669
293500000 5.251063106113743
293600000 5.020037386352484
293700000 4.810098072560358
293800000 5.135397083643896
293900000 5.2781035988934475
294000000 4.9

319400000 2.0592824138627175
319500000 1.9861491984406914
319600000 2.3682624018382734
319700000 2.270471483370896
319800000 2.0648332229607265
319900000 2.5766936264282227
320000000 2.5685457296629286
320100000 2.2770728076821025
320200000 2.15445005774518
320300000 2.0749143533495844
320400000 3.1624011034616717
320500000 2.274110081273178
320600000 2.1875780304914216
320700000 2.181464047095202
320800000 2.65522368322625
320900000 0.6323938718416453
321000000 0.4449115157993449
321100000 1.3416386314078497
321200000 2.5729102418952285
321300000 2.057829320539631
321400000 2.2699075974454237
321500000 2.242587772769284
321600000 2.2111268365373054
321700000 3.1814129565968683
321800000 2.282260616440479
321900000 2.3061505777805196
322000000 3.148690895016381
322100000 3.0969724208198786
322200000 2.54636444124744
322300000 2.7484420133071845
322400000 3.204932864225244
322500000 2.655625527989407
322600000 2.4938841085692456
322700000 2.4980381790399613
322800000 2.382429622663758
3

348000000 1.4692179089875697
348100000 1.4018237278334205
348200000 1.3082851431251705
348300000 1.2334086049533326
348400000 1.4645891201749366
348500000 1.360115050972684
348600000 1.322672558975202
348700000 1.4012253386744307
348800000 0.5160050940995753
348900000 0.33269827314121253
349000000 0.9136381606780237
349100000 1.7301141628408077
349200000 1.5934703231607807
349300000 1.3150731332475643
349400000 1.592482014646089
349500000 1.2281648575129833
349600000 1.3772586634181274
349700000 1.8397148379196864
349800000 1.3721603348824487
349900000 1.937915965034756
350000000 1.5302027890252592
350100000 1.5563271292737106
350200000 1.3239242199953138
350300000 1.3018713471978267
350400000 1.202898601770309
350500000 1.2723508605658507
350600000 1.2965903684405864
350700000 1.327229686760109
350800000 2.280974526499026
350900000 1.017926562942791
351000000 0.32260013427587614
351100000 0.3218024892201079
351200000 2.5322782914074162
351300000 1.3784535396444002
351400000 1.31097679

376000000 0.3088143767315639
376100000 0.30950449717857836
376200000 0.31013104745386044
376300000 0.3077235101630588
376400000 0.3066907937406262
376500000 0.3034995866809964
376600000 0.3040400656319737
376700000 0.30582462496368285
376800000 0.3061981008950995
376900000 0.3056097462626749
377000000 0.3052982568393231
377100000 0.30258313395409187
377200000 0.3004392341714899
377300000 0.2989395403825621
377400000 0.29736456541747447
377500000 0.3008026923585382
377600000 0.2979688877203087
377700000 0.29729126133526296
377800000 0.3054312190278259
best so far: 0
type: [1, 1, 3, 1, 43] 129
cases of this type: 30769209
377900000 1.9950339269061328
378000000 2.024187864433903
378100000 1.9775961625597052
378200000 2.268749404176042
378300000 2.0209668584655343
378400000 1.9215051429662309
378500000 1.957450577398024
378600000 2.4441198144776606
378700000 1.882084300037027
378800000 1.8813489058970811
378900000 2.177416674393089
379000000 2.163048460181074
379100000 1.7924322093467115
3

404100000 0.46317106382865314
404200000 0.5445524648569399
404300000 0.5453176265749852
404400000 0.7220912493150281
404500000 0.7444062265776892
404600000 0.6421115298869132
404700000 0.7682852972640931
404800000 0.5305650889730857
404900000 0.6194177761610866
405000000 0.48623351889778177
405100000 0.657837089766711
405200000 0.6025629260662
405300000 0.4392890956973069
405400000 0.6643670110210214
405500000 0.6095362914163112
405600000 0.48318240679564145
405700000 0.5744450909859432
405800000 0.6881834803023041
405900000 0.5539977974596771
406000000 0.382917320331292
406100000 0.6641555642803192
406200000 0.5912945719109145
406300000 0.4848855828120132
406400000 0.5982988660092334
406500000 0.7005324261461822
406600000 0.6019449966302475
406700000 0.5874432328533232
406800000 0.6110755152959942
406900000 0.6182833288015757
407000000 0.46321032765721876
407100000 0.43998209018871787
407200000 0.6892112395204955
407300000 0.6061332929213762
407400000 0.31674019446442386
407500000 0.3